# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1o2lpxoi9heyQV1hIlsHXWSfDkBPtze-V?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [22]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# abrir o navegador
navegador = webdriver.Chrome()

navegador.get("https://www.google.com.br/")

# passo 1 - pegar a cotação do dólar
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

# passo 2 - pegar a cotação do euro
navegador = webdriver.Chrome()

navegador.get("https://www.google.com.br/")

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
# passo 3 - pegar a cotação do ouro

navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
navegador.quit()

print(cotacao_ouro)

280.47


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [16]:
# passo 4 - importar a lista de produtos

import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")

display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [24]:
# passo 5 - recaucular o preço de cada produto

# atualização cotacao

tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# atualizar o preço base reais(preço base original X cotação)
tabela["Preço Base Reais"] = tabela["Preço Base Original"] * tabela["Cotação"]

# atualizar o preço final (preço base reais x margem)
tabela["Preço Final"] = tabela["Preço Base Reais"] * tabela["Margem"]
# passo 6 - salvar os novos preços dos produtos

display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,4.91490,4914.850851,1.40,6880.791191
1,Carro Renault,4500.00,Euro,5.86323,26384.534798,2.00,52769.069595
2,Notebook Dell,899.99,Dólar,4.91490,4423.360851,1.70,7519.713447
3,IPhone,799.00,Dólar,4.91490,3927.005100,1.70,6675.908670
4,Carro Fiat,3000.00,Euro,5.86323,17589.689865,1.90,33420.410743
5,Celular Xiaomi,480.48,Dólar,4.91490,2361.511152,2.00,4723.022304
6,Joia 20g,20.00,Ouro,280.47000,5609.400000,1.15,6450.810000


### Agora vamos exportar a nova base de preços atualizada

In [25]:
tabela.to_excel("Produtos_novo.xlsx", index=False)